In [1]:
import pandas as pd
from pydantic import BaseModel, TypeAdapter
from PandasToPydantic.src.dataframeSerializer import expandAnnotation, getBaseFields, getListFields, serializeDataframe


In [8]:
class Foo(BaseModel):
    a: int = 123
    #: The sibling of `Foo` is referenced by string
    sibling: 'Foo' = None

In [18]:
class GrandChildModel(BaseModel):
    grandChildString: str
    grandChildInteger: int


class ChildModel(BaseModel):
    childString: str
    childInteger: int
    childListGrandChild: list[GrandChildModel]


class ParentModel(BaseModel):
    parentString: str
    parentInteger: int
    parentFloat: float
    parentListChild: list[ChildModel]

In [14]:
ParentModel.__annotations__

{'parentString': str, 'parentInteger': int, 'parentFloat': float}

In [16]:
getBaseFields(expandAnnotation(ParentModel))

['parentString', 'parentInteger', 'parentFloat']

In [17]:
getBaseFields(ParentModel.__annotations__)

['parentString', 'parentInteger', 'parentFloat']

In [11]:
hasattr(ParentModel, "__annotations__")

True

In [12]:
class DataPoint(BaseModel):
    DataName: str
    DataType: str
    DataValue: str

class DataCollection(BaseModel):
    CollectionName: str
    CollectionDescription: str
    DataPointList: list[DataPoint]

class CollectionOperation(BaseModel):
    OperationName: str
    OperationUser: str
    OperationDate: str
    DataCollectionList: list[DataCollection]

collectionOperationList = TypeAdapter(list[CollectionOperation])

In [13]:
str.__base__ == BaseModel

False

In [14]:
CollectionOperation.__base__ == BaseModel

True

In [15]:
data = pd.read_csv("PandasToPydantic/test/testData.csv")

In [16]:
serializedData = serializeDataframe(data, expandAnnotation(CollectionOperation))


In [17]:
collectionOperationList.validate_python(serializedData)

[CollectionOperation(OperationName='Operation 1', OperationUser='User 1', OperationDate='2023-01-01', DataCollectionList=[DataCollection(CollectionName='Collection 1', CollectionDescription='Description 1', DataPointList=[DataPoint(DataName='Name 1', DataType='Type 1', DataValue='Value 2'), DataPoint(DataName='Name 2', DataType='Type 2', DataValue='Value 1'), DataPoint(DataName='Name 3', DataType='Type 2', DataValue='Value 1')]), DataCollection(CollectionName='Collection 2', CollectionDescription='Description 2', DataPointList=[DataPoint(DataName='Name 1', DataType='Type 1', DataValue='Value 2'), DataPoint(DataName='Name 2', DataType='Type 2', DataValue='Value 1')])]),
 CollectionOperation(OperationName='Operation 2', OperationUser='User 2', OperationDate='2023-02-02', DataCollectionList=[DataCollection(CollectionName='Collection 1', CollectionDescription='Description 1', DataPointList=[DataPoint(DataName='Name 1', DataType='Type 1', DataValue='Value 2'), DataPoint(DataName='Name 2', D

In [18]:
expandAnnotation(CollectionOperation)["DataCollectionList"]

[{'CollectionName': str,
  'CollectionDescription': str,
  'DataPointList': [{'DataName': str, 'DataType': str, 'DataValue': str}]}]